In [ ]:
import sys

sys.path.append("..")

### Test

In [ ]:
from typing import List, Dict
from pathlib import Path
from PIL import Image
from segment.utils import resize_image_pil
from segment.dino_script import get_dino_results
from segment.sam_script import get_sam_results
from segment.utils import get_device
from segment.sam_results import SAMResults
from diffusers.utils import load_image
from segment.sam_script import get_sam_results

DEVICE = get_device()


def load_resize_image(image_path):
    image_pil = load_image(image_path).convert("RGB")
    image_pil = resize_image_pil(image_pil, 1024)
    return image_pil


def get_masks(
    text_prompt, image_paths: List[str], labels_dict: Dict[str, int], **kwargs
) -> SAMResults:
    device = get_device()

    images = [load_resize_image(url) for url in image_paths]

    # Get the boxes from the prompts using DINO
    global dino_results
    dino_results = get_dino_results(images, text_prompt, device, **kwargs)

    # Get the masks from the images and boxes using SAM
    unformatted_results = get_sam_results(images, dino_results, text_prompt, device)

    sam_results = SAMResults(
        images[0], labels_dict, **unformatted_results[0], person_masks_only=False,**kwargs
    )
    return sam_results

In [ ]:
text_prompt = ["hair", "face", "neck", "arm", "hand", "back", "leg", "clothing", "phone", "hat", "foot"]

text_prompt = ".".join(text_prompt)

labels_dict = {prompt.strip(): idx for idx, prompt in enumerate(text_prompt.split("."))}


url = "https://i.pinimg.com/originals/68/82/ce/6882cefe21f75234697e8508eb6a3232.jpg"
url = "/Users/jordandavis/Downloads/hailey.webp"

sam_results = get_masks(text_prompt, [url], labels_dict, box_threshold=0.3, text_threshold=0.25)

sam_results.display_results()

In [ ]:
import numpy as np 

sam_scores = np.array(sam_results.scores)

dino_scores = dino_results.scores[0].cpu().numpy()

np.mean([sam_scores, dino_scores], axis=0)


In [6]:
dino_results.boxes

[tensor([[ 393.4577,  119.5898,  648.7547,  424.5431],
         [ 224.5373,  440.3598,  805.0222, 1022.4713]])]